In [1]:
import numpy as np
import matplotlib.pyplot as plt
# import plotly.express as px
import plotly.graph_objects as go
import pandas as pd
from plotly.offline import init_notebook_mode, iplot
import plotly.io as pio
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
from torchvision.utils import save_image
import datetime
import os
import sys
from tqdm.notebook import tqdm

In [2]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=(0.5,), std=(0.5, ))
])

In [3]:
train_dataset = torchvision.datasets.MNIST(root=".", transform=transform)

In [4]:
batch_size = 128
train_iter = torch.utils.data.DataLoader(train_dataset, shuffle=True, 
                                        batch_size=batch_size)

In [5]:
def get_discriminator_block(input_dim, output_dim):
    """
    Description : To create a discriminator neural network block
    
    Parameters :
    @param input_dim -- a python integer for the input dimension
    @param output_dim -- a python integer for the output dimension
    
    Return :
    @ret neural_block -- a Sequential layer
    """
    
    neural_block = nn.Sequential(nn.Linear(input_dim, output_dim),
                         nn.LeakyReLU(negative_slope=0.2))
    
    return neural_block

In [6]:
def get_generator_block(input_dim, output_dim):
    """
    Description : To create a generator neural network block
    
    Parameters:
    @param input_dim -- a python integer for the input dimension
    @param output_dim -- a python integer for the output dimension
    
    Return :
    @ret neural_block -- a Sequential Layer
    """
    
    neural_block = nn.Sequential(nn.Linear(input_dim, output_dim),
                                nn.BatchNorm1d(output_dim, momentum=0.7),
                                nn.LeakyReLU(negative_slope=0.2))
    
    return neural_block

In [7]:
class Discriminator(nn.Module):
    
    def __init__(self, input_dim, output_dim, hidden_dim=128):
        
        super(Discriminator, self).__init__()
        
        self.disc = nn.Sequential(get_discriminator_block(input_dim, hidden_dim*4),
                                 get_discriminator_block(hidden_dim*4, hidden_dim*2),
                                 nn.Linear(hidden_dim*2, output_dim))
        
    
    def forward(self, X):
        out = self.disc(X)
        return out

In [8]:
class Generator(nn.Module):
    
    def __init__(self, input_dim, output_dim, hidden_dim=128):
        super(Generator, self).__init__()
        
        self.gen = nn.Sequential(get_generator_block(input_dim, hidden_dim*2),
                                get_generator_block(hidden_dim*2, hidden_dim*4),
                                nn.Linear(hidden_dim*4, output_dim),
                                nn.Tanh())
        
    def forward(self, X):
        out = self.gen(X)
        return out
        

In [9]:
def generate_noise(batch_size, latent_dim=100):
    """
    Description : Function to generate noise from Gaussian Distribution
    
    Parameters : 
    @param batch_size -- a python integer representing the batch size
    @param latent_dim -- a python integer representing the dimension of the noise
    
    Return :
    Random sampling of data from Gaussian Distribution as per the size
    """
    return torch.randn(batch_size, latent_dim)

In [10]:
criterion = nn.BCEWithLogitsLoss()
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
latent_dim = 100

In [11]:
G = Generator(latent_dim, 784).to(device)
D = Discriminator(784, 1).to(device)

g_optim = torch.optim.Adam(G.parameters(), lr=1e-4 * 2)
d_optim = torch.optim.Adam(D.parameters(), lr=1e-4 * 2)

In [12]:
def discriminator_loss(G, D, criterion, real, \
                       batch_size, latent_dim,device):
    """
    Description : Function to calculate the loss for the discriminator
    
    Parameters:
    @param G -- The Generator Network
    @param D -- The Discriminator Network
    @param criterion -- The loss function
    @param real -- a numpy array representing the real images
    @param batch_size -- python integer representing the batch_size
    @param latent_dim -- python integer representing the latent dimension
    @param device -- whether 'cpu' or 'cuda:0'
    
    Return :
    @ret disc_loss -- the Discriminator Loss
    """
    
    zeros_ = torch.zeros(batch_size, 1).to(device)
    ones_ = torch.ones(batch_size, 1).to(device)
    
    real_loss = criterion(D(real), ones_)
    
    noise = generate_noise(batch_size, latent_dim).to(device)
    fake_img = G(noise).detach()
    
    fake_loss = criterion(D(fake_img), zeros_)
    
    disc_loss = (fake_loss + real_loss) * 0.5
    
    return disc_loss

In [13]:
def generator_loss(G, D, criterion, batch_size, latent_dim, device):
    
    """
    Description : Function to calculate the generator loss
    
    Parameters:
    @param G -- The Generator Network
    @param D -- The Discriminator Network
    @param criterion -- the Loss function
    @param batch_size -- the batch size
    @param latent_dim -- the latent dimension
    @param device -- whether 'cpu' or 'cuda:0'
    
    Return:
    @ret gen_loss -- the Generator Loss
    """
    
    noise = generate_noise(batch_size, latent_dim).to(device)
    ones_ = torch.ones(batch_size, 1).to(device)
    
    fake_image = G(noise)
    
    gen_loss = criterion(D(fake_image), ones_)
    
    return gen_loss

In [14]:
if not os.path.exists("./IMAGES/SimpleGANs"):
    os.mkdir("./IMAGES/SimpleGANs")

In [15]:
def scale_image(img):
    return (img + 1) / 2

In [16]:
def train(G, D, criterion, g_optim, d_optim, data_iter, latent_dim, epochs=200, save=True):
    """
    Description : To train the GANs model
    
    Parameters:
    @param G -- The Generator Network
    @param D -- The Discriminator Network
    @param g_optim -- the Generator Optimizer
    @param d_optim -- the Discriminator Optimizer
    @param data_iter -- the data to train on
    @param latent_dim -- the dimension for the noise
    @param epochs -- number of epochs to run (default=200)
    @param save -- whether to save the images or not (default=True)
    
    Return :
    @ret g_losses -- the Generator Losses
    @ret d_losses -- the Discriminator Losses
    """
    g_losses = []
    d_losses = []
    
    for epoch in range(epochs):
        
        d_loss = []
        g_loss = []
        batch_size = 0
        for inputs, _ in tqdm(data_iter):
            
            batch_size = inputs.size(0)
            inputs = inputs.resize(batch_size, 784).to(device)
            
            
            ##############################################
            ########### TRAIN DISCRIMINATOR ############
            #############################################
            d_optim.zero_grad()
            
            dLoss = discriminator_loss(G, D, criterion, inputs, batch_size,\
                                      latent_dim, device) 
            
            dLoss.backward()
            d_optim.step()
            
            d_loss.append(dLoss.item())
            
            ##############################################
            ########### TRAIN GENERATOR ############
            #############################################
            gLoss = []
            for _ in range(2):
                g_optim.zero_grad()
                _gLoss = generator_loss(G, D, criterion, batch_size, latent_dim, device)

                _gLoss.backward()
                g_optim.step()
                gLoss.append(_gLoss.item())
            g_loss.append(np.mean(gLoss))
            
        
        d_loss = np.mean(d_loss)
        g_loss = np.mean(g_loss)
        
        g_losses.append(g_loss)
        d_losses.append(d_loss)
        
        print(f"Epoch:{epoch+1}/{epochs} || Disc Loss: {d_loss} || Gen Loss : {g_loss}")
        
        if save:
            noise = generate_noise(batch_size, latent_dim)
            fake_img = G(noise.to(device))
            fake_img = fake_img.reshape(-1, 1, 28, 28)
            save_image(scale_image(fake_img), f"./IMAGES/SimpleGANs/gan_{epoch}.png")
            
    return g_losses, d_losses
            
            
            
            
            
            
            
            

In [17]:
g_losses, d_losses = train(G, D, criterion, g_optim, d_optim, train_iter, latent_dim)

/home/adityam/anaconda3/lib/python3.8/site-packages/torch/tensor.py:358: UserWarning:

non-inplace resize is deprecated




Epoch:1/200 || Disc Loss: 0.25948005675602315 || Gen Loss : 3.5365867766934924



Epoch:2/200 || Disc Loss: 0.22322057572000825 || Gen Loss : 4.065337242602285



Epoch:3/200 || Disc Loss: 0.3088390910104394 || Gen Loss : 4.110610055770955



Epoch:4/200 || Disc Loss: 0.2643479453697642 || Gen Loss : 3.9793219167286398


...


Epoch:196/200 || Disc Loss: 0.6337229205347074 || Gen Loss : 0.9089722222229565



Epoch:197/200 || Disc Loss: 0.6436653449845466 || Gen Loss : 0.8843934754572952



Epoch:198/200 || Disc Loss: 0.637661492773719 || Gen Loss : 0.8883445601600574



Epoch:199/200 || Disc Loss: 0.6377832948017731 || Gen Loss : 0.8895135456438004



Epoch:200/200 || Disc Loss: 0.6374745320663777 || Gen Loss : 0.8913704658240906


In [21]:
# fig = go.Figure(data = [
#     go.Scatter(name="Discriminator", x=[i for i in range(len(d_losses))], y=d_losses),
#     go.Scatter(name="Generator", x=[i for i in range(len(g_losses))], y=g_losses)
# ])
fig = go.Figure()
fig.add_trace(go.Scatter(name="Discriminator", x=[i for i in range(len(d_losses))], 
                         y=d_losses))

fig.add_trace(go.Scatter(name="Generator", x=[i for i in range(len(g_losses))], 
                         y=g_losses))

fig.update_layout(title=" Epochs vs Losses",
                 xaxis_title="Epochs",
                 yaxis_title="Loss", )
fig.show()